<center><img src='https://urldefense.com/v3/__https://sedigitalylanzate.es/wp-content/uploads/2019/03/logoeoiok-e1552040050697.png__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRMSMR5lA$ ' width="500"></center>

# Ejercicio de algoritmo de filtro colaborativo

El objetivo de este ejercicio es generar un algoritmo de filtro colaborativo con los datos del dataset beer_review. La información se corresponde con la reviews de la web https://urldefense.com/v3/__https://www.beeradvocate.com/__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRXm04eLI$ 

<center><img src='https://urldefense.com/v3/__https://www.pngitem.com/pimgs/m/39-394080_beer-advocate-hd-png-download.png__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRskeByAY$ ' width=600 ></center>

## Información

Este dataset consiste en revisiones de cerveza de Beeradvocate. Los datos abarcan un período de más de 10 años, que incluye todas ~ 1.5 millones de revisiones hasta noviembre de 2011. Cada revisión incluye calificaciones en términos de cinco "aspectos": apariencia, aroma, paladar, sabor e impresión general. Las revisiones incluyen información del producto y del usuario, seguida de cada una de estas cinco clasificaciones, y una revisión en formato texto. 

Para nuestro ejercicio vamos a usar una versión reducida de este dataset, que contiene las reviews de aquellos usuarios que han valorado al menos 50 cervezas.

## Pasos a seguir

1. Descarga la información de beeradvocate en una carpeta que se llame `datos_ejercicios/beeradvocate` y que se encuentre al mismo nivel que el notebook en el que vas a ejecutar el código.

2. Lee el fichero `beer_reviews_50.csv` y crea dos datasets:
    - uno con las reviews de los usuarios.
    - otro con la información de la cerveza (brewery, beer_name, beer_style)
    
3. Divide la información en los conjuntos de train y test.

4. De los distintos tipos de algoritmos que conocemos elige uno para generar la recomendación. Si creas más de uno explica el motivo de tu elección.


5. Crea una función que dado un usuario nos devuelva el top 10 de recomendaciones. 

6. Genera un html con los resultados de notebook file -> Export Notebook As... -> Export Notebook as HTML. Este html será el documento que tendrás que entregar para la evaluación del ejercicio.

In [330]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [401]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import seaborn as sns

In [402]:
totalDB = pd.read_csv('/content/drive/MyDrive/datos_ejercicios/beeradvocate/beer_reviews_50.csv', low_memory=False)

In [403]:
totalDB.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,user_id
0,163,Amstel Brouwerij B. V.,1023302870,3.5,3.5,3.5,Mar-77,Light Lager,3.0,3.0,Amstel Light,3.50,436,1
1,395,Bluegrass Brewing Co. - East St. Matthew's,1055007422,4.5,4.0,4.5,Mar-77,American Pale Ale (APA),3.5,4.0,American Pale Ale,5.79,1122,1
2,395,Bluegrass Brewing Co. - East St. Matthew's,1055009536,5.0,3.5,4.5,Mar-77,English Porter,3.5,4.5,Dark Star Porter,5.60,1123,1
3,365,Pike Pub & Brewery,1052070806,5.0,4.5,4.5,Mar-77,American Stout,4.0,4.5,Pike Street XXXXX Stout,7.00,966,1
4,1199,Founders Brewing Company,1065491961,4.5,4.5,4.0,Mar-77,Scotch Ale / Wee Heavy,3.0,4.0,Founders Dirty Bastard,8.50,7463,1


In [404]:
reviews = totalDB[['user_id','review_profilename','review_overall','review_aroma','review_appearance','review_palate','review_taste','beer_abv']]

In [405]:
cervezas = totalDB[['beer_beerid','brewery_name','beer_style','beer_name']]

In [406]:
reviews.head()

,user_id,review_profilename,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
0,1,Mar-77,3.5,3.5,3.5,3.0,3.0,3.50
1,1,Mar-77,4.5,4.0,4.5,3.5,4.0,5.79
2,1,Mar-77,5.0,3.5,4.5,3.5,4.5,5.60
3,1,Mar-77,5.0,4.5,4.5,4.0,4.5,7.00
4,1,Mar-77,4.5,4.5,4.0,3.0,4.0,8.50


In [477]:
cervezas.head

<bound method NDFrame.head of         beer_beerid  ...                         beer_name
0               436  ...                      Amstel Light
1              1122  ...                 American Pale Ale
2              1123  ...                  Dark Star Porter
3               966  ...           Pike Street XXXXX Stout
4              7463  ...            Founders Dirty Bastard
...             ...  ...                               ...
256591          694  ...                       Don de Dieu
256592           31  ...                Blanche De Chambly
256593        14732  ...                      Edition 2004
256594           33  ...                           Maudite
256595        26072  ...  Noire De Chambly / Chambly Noire

[256596 rows x 4 columns]>

In [408]:
U_rating = totalDB[['user_id','beer_beerid','review_overall']] 

In [409]:
U_rating.head()

,user_id,beer_beerid,review_overall
0,1,436,3.5
1,1,1122,4.5
2,1,1123,5.0
3,1,966,5.0
4,1,7463,4.5


In [410]:
conteo_rating = (
    U_rating[['user_id','beer_beerid']]
    .groupby('user_id')
    .count().reset_index()
    .rename(columns={'beer_beerid': 'count'})
)

In [411]:
minimo_votos = conteo_rating['count'].quantile(0.70)

In [412]:
minimo_votos

147.0

In [413]:
lista_votos = conteo_rating[conteo_rating['count']>= minimo_votos].user_id.tolist()

In [414]:
U_rating = U_rating[U_rating.user_id.isin(lista_votos)]

In [415]:
from sklearn.model_selection import train_test_split


X = U_rating.copy()
y = U_rating['user_id']

In [416]:
train_test_split

<function sklearn.model_selection._split.train_test_split>

In [417]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [418]:
from sklearn.metrics import mean_squared_error

In [419]:
mean_squared_error

<function sklearn.metrics._regression.mean_squared_error>

In [420]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [421]:
def baseline(usuario,cerveza):
    return 3.0

In [422]:
pares = zip(X_test['user_id'], X_test['beer_beerid'])

In [423]:
def calculo_rmse(modelo):
    
    #Creamos una tupla de usuarios-cervezas del conjunto de test
    pares = zip(X_test['user_id'], X_test['beer_beerid'])
    
    #Hacemos una predicción para cada tupla
    y_pred = np.array([modelo(user_id, beer_beerid) for (user_id, beer_beerid) in pares])
    
    #Sacamos el dato real
    y_true = np.array(X_test['review_overall'])
    
    #devolvemos el valor de rmse
    return rmse(y_true, y_pred)

    

In [454]:
calculo_rmse(baseline)

1.0500646154202502

In [425]:
usuario_ejemplo = int(reviews['user_id'].sample(n=1, random_state=1))

In [426]:
usuario_ejemplo

808

In [427]:
puntuaciones_ejemplo = (
    U_rating[U_rating.user_id == usuario_ejemplo]
    .sort_values("review_overall", ascending=False)
    .merge(
        cervezas[["beer_beerid", "beer_name"]],
        on='beer_beerid'
    )
)

In [428]:
puntuacion_ejemplo = puntuaciones_ejemplo.drop_duplicates()

In [429]:
puntuacion_ejemplo.head(10)

,user_id,beer_beerid,review_overall,beer_name
0,808,771,5.0,Entire Butt English Porter
122,808,473,5.0,Dunster Street Amber
124,808,473,4.0,Dunster Street Amber
126,808,6171,5.0,Best Bitter
129,808,666,5.0,Hacker-Pschorr Dunkle Weisse
316,808,39201,5.0,Hopworks IPA
397,808,477,5.0,Irish Dry Stout
411,808,18595,5.0,Irish Stout
418,808,1818,5.0,Waas Klokbier
421,808,638,5.0,Blackout Oatmeal Stout


In [430]:
pd.options.display.max_rows = puntuacion_ejemplo.shape[0]

In [431]:
puntuacion_ejemplo.sort_values(["review_overall", "beer_beerid"], ascending=False).head(10)

,user_id,beer_beerid,review_overall,beer_name
316,808,39201,5.0,Hopworks IPA
411,808,18595,5.0,Irish Stout
3314,808,8784,5.0,Hirsch Bavarian-Weissbier Hefeweisse
126,808,6171,5.0,Best Bitter
2600,808,4064,5.0,Brakspear Bitter
3368,808,1871,5.0,Dobbelen Bruinen
418,808,1818,5.0,Waas Klokbier
2640,808,1658,5.0,Big Bear Black Stout
0,808,771,5.0,Entire Butt English Porter
129,808,666,5.0,Hacker-Pschorr Dunkle Weisse


In [432]:
vistas_ejemplo = puntuacion_ejemplo.beer_beerid.tolist()

In [460]:
matriz = X_train.pivot_table(values='review_overall', index='beer_beerid', columns='user_id')

In [479]:
matriz.head

<bound method NDFrame.head of user_id      1     8     10    13    16    ...  1814  1817  1824  1827  1828
beer_beerid                                ...                              
3             NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
4             NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
11            3.0   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   2.0   NaN
12            NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   3.5   NaN
13            3.5   NaN   NaN   NaN   NaN  ...   3.0   NaN   NaN   1.5   NaN
...           ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...
77195         NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
77197         NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
77281         NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
77303         NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN
77307         NaN   NaN   NaN   NaN   NaN  ...

In [480]:
def fc_media(usuario,cerveza):
    
    #Comprobamos que la película esté en la sparse-matrix
    if cerveza in matriz:
        #Calculamos la media
        media = matriz[cerveza].mean()
    
    else:
        #En caso de no tenerla devolvemos un 3
        media = 3.0
    
    return media

In [481]:
calculo_rmse(fc_media)

1.039023735446685

In [482]:
recomendaciones_media = []
for i in cervezas.beer_beerid.tolist():
    
    recomendaciones_media.append([i, fc_media(usuario_ejemplo, i )])
    
    recos = (
        pd.DataFrame(recomendaciones_media,
                     columns=["beer_beerid", "output"]
                    )
        .sort_values("output", ascending=False)
        .merge(
            cervezas[["beer_beerid", "beer_name"]],
            on='beer_beerid'
        )
    
    )
    recos = recos[~recos.beer_beerid.isin(vistas_ejemplo)].head(20)

KeyboardInterrupt: ignored

In [483]:
recos

,beer_beerid,output,beer_name
32,933,4.03125,Blue Paddle Pilsner
33,933,4.03125,Blue Paddle Pilsner
34,933,4.03125,Blue Paddle Pilsner
35,933,4.03125,Blue Paddle Pilsner
36,933,4.03125,Blue Paddle Pilsner
37,933,4.03125,Blue Paddle Pilsner
38,933,4.03125,Blue Paddle Pilsner
39,933,4.03125,Blue Paddle Pilsner
40,933,4.03125,Blue Paddle Pilsner
41,933,4.03125,Blue Paddle Pilsner
